In [1]:
! pip install -U transformers
!pip install sacremoses datasets evaluate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.5 MB/s eta 0:0

In [2]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, BioGptForSequenceClassification, get_linear_schedule_with_warmup, set_seed, AutoModel
from tqdm import tqdm
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
torch.manual_seed(42)

from peft import (
    get_peft_model,
    LoraConfig,
    PeftType,
)

from datasets import load_dataset, Dataset, DatasetDict
import datasets
import evaluate
import pandas as pd
import numpy as np

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
tokenizer.padding_side = "left"

model = BioGptForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=2, return_dict=True)
#model.resize_token_embeddings(len(tokenizer))

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
peft_type = PeftType.LORA
num_epochs = 20
batch_size = 32

# Define LoRA Config
peft_config  = LoraConfig(
 r=8,
 lora_alpha=16,
 inference_mode=False,
 lora_dropout=0.1,
 target_modules= ["k_proj", 'v_proj', 'q_proj']
 )
lr = 3e-4
#3e-4
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
#model

trainable params: 1,179,648 || all params: 347,944,960 || trainable%: 0.33903293210512375


In [5]:
dataset = load_dataset("csv", data_files="---")

dataset = dataset.rename_column("Chief Complaint", "Text")
dataset = dataset.rename_column("Consensus", "labels")

df_train = pd.DataFrame( dataset['train'] )
df_train = df_train[~df_train["labels"].isin(["-", "U"])].reset_index(drop=True)
dataset = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train)
    }
)

dataset = dataset.class_encode_column('labels')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Casting to class labels:   0%|          | 0/188 [00:00<?, ? examples/s]

In [6]:
train_testvalid = dataset['train'].train_test_split(test_size=0.4, seed=42, stratify_by_column="labels")
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42, stratify_by_column="labels")
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [7]:
# Convert to a pandas DataFrame
df_test = pd.DataFrame( dataset['test'] )
df_train = pd.DataFrame( dataset['train'] )
df_valid = pd.DataFrame( dataset['valid'] )

print(df_train.groupby("labels").size())
print(df_valid.groupby("labels").size())
print(df_test.groupby("labels").size())


# Convert back to dataset
dataset = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train),
        "test": Dataset.from_pandas(df_test),
        "valid": Dataset.from_pandas(df_valid),
    }
)

labels
0    70
1    42
dtype: int64
labels
0    24
1    14
dtype: int64
labels
0    24
1    14
dtype: int64


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["Text"])
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["Text", "Predict"])

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

In [9]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")
# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_dataset["valid"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size)
test_dataloader = DataLoader(tokenized_dataset["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size)

In [10]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [11]:
metric = evaluate.load('bstrai/classification_report')
metric2 = evaluate.load('hyperml/balanced_accuracy')

In [12]:
#rsults without fine tunning
model.to(device)
for step, batch in enumerate(tqdm(test_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )
    metric2.add_batch(
        predictions=predictions,
        references=references,
    )


eval_metric = metric.compute()
eval_metric2 = metric2.compute()

print()
print()
print("results on the test data, before fine tunning, 2019, 2 labels, biogpt:")
print(eval_metric)
print(eval_metric2)

100%|██████████| 2/2 [00:22<00:00, 11.37s/it]



results on the test data, before fine tunning, 2019, 2 labels, biogpt:
{'0': {'precision': 0.8888888888888888, 'recall': 0.3333333333333333, 'f1-score': 0.48484848484848486, 'support': 24}, '1': {'precision': 0.4482758620689655, 'recall': 0.9285714285714286, 'f1-score': 0.6046511627906977, 'support': 14}, 'accuracy': 0.5526315789473685, 'macro avg': {'precision': 0.6685823754789272, 'recall': 0.6309523809523809, 'f1-score': 0.5447498238195914, 'support': 38}, 'weighted avg': {'precision': 0.7265577737447066, 'recall': 0.5526315789473685, 'f1-score': 0.528986313564037, 'support': 38}}
{'balanced_accuracy': 0.6309523809523809}


In [13]:
import copy


best_f1 = 0.0
best_epoch = -1
best_model = None

model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric2.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric2 = metric2.compute()
    print(f"epoch {epoch}:", eval_metric2)

    # Check if the current F1 score is better than the previous best
    if eval_metric2["balanced_accuracy"] > best_f1:
        best_f1 = eval_metric2["balanced_accuracy"]
        best_epoch = epoch
        print("new best model found, saving...")

        # Save the model checkpoint with the best F1 score
        best_model = copy.deepcopy(model)

 50%|█████     | 2/4 [01:14<01:14, 37.41s/it]


KeyboardInterrupt: ignored

In [ ]:
best_model.push_to_hub("Lukee4/biogpt-2019_2labels", use_auth_token=True)


adapter_model.bin:   0%|          | 0.00/4.78M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Lukee4/biogpt-2019/commit/cd5b95f9e814257657e981e29d31c6ccb84809ea', commit_message='Upload model', commit_description='', oid='cd5b95f9e814257657e981e29d31c6ccb84809ea', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
for step, batch in enumerate(tqdm(test_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = best_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )
    metric2.add_batch(
        predictions=predictions,
        references=references,
    )


eval_metric = metric.compute()
eval_metric2 = metric2.compute()

print()
print()
print("results on the test data, after fine tunning, 2019, 2 labels, biogpt:")
print(eval_metric)
print(eval_metric2)

100%|██████████| 2/2 [00:00<00:00,  2.89it/s]



results on the test data:
{'0': {'precision': 0.7037037037037037, 'recall': 0.7916666666666666, 'f1-score': 0.7450980392156864, 'support': 24}, '1': {'precision': 0.5454545454545454, 'recall': 0.42857142857142855, 'f1-score': 0.4799999999999999, 'support': 14}, 'accuracy': 0.6578947368421053, 'macro avg': {'precision': 0.6245791245791246, 'recall': 0.6101190476190476, 'f1-score': 0.6125490196078431, 'support': 38}, 'weighted avg': {'precision': 0.6454013822434875, 'recall': 0.6578947368421053, 'f1-score': 0.6474303405572757, 'support': 38}}
{'balanced_accuracy': 0.6101190476190476}
